# Multitask error vs correct classification


In [ ]:
# you have to install openneuro-py and mne-bids packages
# if using colab, run:
# !pip install openneuro-py
# !pip install mne-bids
# if working locally, install modlules using pip

import os
import os.path as op
import openneuro

from mne.datasets import sample
from mne_bids import (
    BIDSPath,
    read_raw_bids,
    print_dir_tree,
    make_report,
    find_matching_paths,
    get_entity_vals,
)
import mne

import numpy as np

## 1. Download dataset

In [ ]:
dataset = "ds004883"

# number of participants to fetch
n = 10
subjects = [f'sub-{subject:03}' for subject in range(1, n)]

# Download one subject's data from each dataset
bids_root = op.join(op.dirname(sample.data_path()), dataset)
if not op.isdir(bids_root):
    os.makedirs(bids_root)

# this download full dataset, it might took few hours
openneuro.download(dataset=dataset, target_dir=bids_root, include=subjects)

## 2. Read data and create epochs

In [ ]:
def read_flanker_data(participants_num=10):
    datatype = "eeg"
    bids_path = BIDSPath(root=bids_root, datatype=datatype)
    
    tasks = [
    "ffa", 
    "ffb", 
    'ffc'
    ]
    subjects = [f'{subject:03}' for subject in range(1, participants_num)]
    suffix = "eeg"

    # define montage to be set
    montage = mne.channels.make_standard_montage('GSN-HydroCel-129')
    
    eeg_data = {}
    for task in tasks:
        for subject_id in subjects:
            # extract subject and task specific path to data
            bids_path = bids_path.update(subject=subject_id, task=task, suffix=suffix) 

            # create raw
            this_raw = read_raw_bids(bids_path=bids_path, verbose=False, extra_params={'preload':True})
        
            ################################
            ### Basic preprocessing
            ################################
        
            # set montage
            this_raw.set_montage(montage)
        
            # filter data
            raw_filtered = this_raw.copy().filter(
                l_freq=.1,
                h_freq=30.0,
                n_jobs=10,
                method='iir',
                iir_params=None,
            )
        
            # Notch filter at 60 Hz
            raw_filtered = raw_filtered.notch_filter(
                freqs=np.arange(60, (raw_filtered.info['sfreq'] / 2), 60),
                n_jobs=10,
            )
        
            # create events
            events, events_ids = mne.events_from_annotations(raw_filtered)
        
            # create epochs
            epochs = mne.Epochs(
                raw_filtered, 
                tmin=-0.2,
                tmax=0.8,
                events=events, 
                event_id= {'cor': 2, 'err': 3},
                baseline=(-0.2, 0)
            )
        
            ################################
            ### Add epochs to dict
            ################################
            
            if task in eeg_data.keys():
                eeg_data[task].append(epochs)
            else:
                eeg_data[task] = [epochs]
    return eeg_data

In [ ]:
data = read_flanker_data(participants_num=n)